# Using Scikit-Learn's `Pipeline()` class for filling missing data and encoding categorical data

This notebook extends the code in the "Putting It All Together" with some more explanations for what's going on behind the scenes.

More specifically, what happens with the `Pipeline()` class as we use it to impute missing values and encode (turn into numbers) categorical values.

The main takeaways:
- Encode and fill your **categorical data** into numbers (on the whole dataset), if necessary.
- Split your data (into train/test), always keep your training & test data separate.
- Fill/transform the **numerical data** on the training set and test sets separately.
- Using [`Pipeline()`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) helps to ensure all of this is done for you.

In [14]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

**Note:** Since this notebook is running on Colab, the data has been imported directly from GitHub. It is the same data used in the videos.

Data on GitHub: https://raw.githubusercontent.com/mrdbourke/zero-to-mastery-ml/master/data/car-sales-extended-missing-data.csv

In [15]:
car_sales_missing = pd.read_csv("https://raw.githubusercontent.com/mrdbourke/zero-to-mastery-ml/master/data/car-sales-extended-missing-data.csv")
car_sales_missing.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


In [16]:
# Check missing values
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [17]:
# Drop the rows with no labels
car_sales_missing.dropna(subset=["Price"], inplace=True)
car_sales_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [18]:
# Split data into X & y
X = car_sales_missing.drop("Price", axis=1)
y = car_sales_missing["Price"]

Now we've dropped the rows with no labels and split our data into `X` and `y`, let's create a [`Pipeline()`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) (or a few of them) to fill the rest of the missing values, encode them if necessary (turn them into numbers) and fit a model to them.

A `Pipeline()` in Scikit-Learn is a class which allows us to put multiple steps, such as filling data and then modelling it, together sequentially.

More specifically, we'll go through the following steps:
1. Define categorical, door and numeric features.
2. Build transformer `Pipeline()`'s for imputing missing data and encoding data.
3. Combine our transformer `Pipeline()`'s with [`ColumnTransformer()`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html).
4. Build a `Pipeline()` to preprocess and model our data with the `ColumnTransformer()` and [`RandomForestRegressor()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html).
5. Split the data into train and test using `train_test_split()`.
6. Fit the preprocessing and modelling `Pipeline()` on the training data.
7. Score the preprocessing and modelling `Pipeline()` on the test data.


Let's start with steps 1. and 2.

We'll build the following:
* A categorical transformer to fill our categorical values with the value `'missing'` and then one encode them.
* A door transformer to fill the door column missing values with the value `4`.
* A numeric transformer to fill the numeric column missing values with the mean of the rest of the column.

All of these will be done with the `Pipeline()` class.

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # this will help us fill missing values
from sklearn.preprocessing import OneHotEncoder # this will help us turn our categorical variables into numbers

# Define categorical columns
categorical_features = ["Make", "Colour"]
# Create categorical transformer (imputes missing values, then one hot encodes them)
categorical_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
  ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Define door feature
door_feature = ["Doors"]
# Create door transformer (fills all door missing values with 4)
door_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='constant', fill_value=4)),
])

# Define numeric features
numeric_features = ["Odometer (KM)"]
# Create a transformer for filling all missing numeric values with the mean
numeric_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='mean'))
])

In [20]:
# The `handle_unknown` parameter in the `OneHotEncoder` is used to specify how the encoder should handle categorical values that were not present in the training data. It has three possible values:

# 1. `'error'` (default): If a new categorical value that was not seen during fit is present at transform time, an error will be raised.

# 2. `'ignore'`: If a new categorical value that was not seen during fit is present at transform time, it will be ignored and not included in the one-hot encoding.

# 3. `'ignore_new_category'`: If a new categorical value that was not seen during fit is present at transform time, a new one-hot feature is created for this new category.

# In your case, `handle_unknown='ignore'` is specified. This means that if there are any categorical values in the data during the `transform` step that were not present in the data during the `fit` step, those values will be ignored and not included in the one-hot encoding.

# This can be useful when you expect that your training data may not contain all possible categories, and you want to handle new, unseen categories gracefully during prediction or transformation, instead of raising an error.

# For example, let's say your training data had categories `['A', 'B', 'C']`, and the `OneHotEncoder` created three columns for these categories. 
# If, during prediction, a new category `'D'` is encountered, and `handle_unknown='ignore'`, the encoder will simply ignore this new category and return the one-hot encoding with the three columns corresponding to `'A'`, `'B'`, and `'C'`.
# If `handle_unknown='error'` (default), the encoder would raise an error in this case.
# The handle_unknown parameter is only relevant when you are encoding categorical features using the OneHotEncoder. It is not needed for imputing missing values in numerical features or filling missing values with a constant for non-categorical features.

Wonderful! Now we've got a way to fill our missing variables and turn them into numbers (one hot encode the categorical variables), let's take care of step 3.

3. Combine our transformer `Pipeline()`'s with `ColumnTransformer()`.


In [21]:
from sklearn.compose import ColumnTransformer

# Create a column transformer which combines all of the other transformers
# into one step
preprocessor = ColumnTransformer(
    transformers=[
      # (name, transformer_to_use, features_to_use transform)
      ('categorical', categorical_transformer, categorical_features),
      ('door', door_transformer, door_feature),
      ('numerical', numeric_transformer, numeric_features)
])

Okay, we've got a `ColumnTransformer()` saved to `preprocessor`, let's make another `Pipeline()` to combine it with a `RandomForestRegressor()` model and take care of step 4.

4. Build a `Pipeline()` to preprocess and model our data with the `ColumnTransformer()` and`RandomForestRegressor()`.

In [22]:
from sklearn.ensemble import RandomForestRegressor

# Create the preprocessing and modelling pipeline
model = Pipeline(steps=[('preprocessor', preprocessor), # this will fill our missing data and make sure it's all numbers
                        ('regressor', RandomForestRegressor())]) # this will model our data

Creating the `model` pipeline like we did is like saying to Scikit-Learn, "Hey, preprocess all of the data we pass you first (using `preprocessor`) and then model it with `RandomForestRegressor()`.

Let's now do step 5 and 6.

5. Split the data into train and test using `train_test_split()`.
6. Fit the preprocessing and modelling `Pipeline()` on the training data.


In [23]:
from sklearn.model_selection import train_test_split

# Split data into train and teset sets
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Fit the model on the training data
# (note: when fit() is called with a Pipeline(), fit_transform() is used for transformers)
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Make', 'Colour']),
                                                 ('door',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=4,
                                                                                 strategy='constant'))]),
                                                  ['Doors']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Odometer (KM)'])])),
                ('regressor', RandomForestRegressor())])

If it all worked, we should've seen a print out of the steps in the modelling `Pipeline()`.

We've got 1 step left and that's to evaluate our model pipeline on the test data.

7. Score the preprocessing and modelling `Pipeline()` on the test data.

In [24]:
# Score the model on the data
# (note: when score() or  predict() is called with a Pipeline(), transform() is used for transformers)
model.score(X_test, y_test)

0.22188417408787875

Nice! Our `Pipeline()` steps seem to have worked for the test dataset as well.

There are a few things going on behind the scenes here. The main ones being how the `Pipeline()` class deals with data transformation.

## Pipeline behind the scenes

When filling **numerical data**, it's important **not** to use values from the test set to fill values in the training set. Since we're trying to predict on the test set, this would be like taking information from the future to fill in the past.

Let's have an example.

In our case, the `Odometer (KM)` column is missing values. We could fill every value in the column (before splitting it into train and test) with the `mean()`. But this would result in using information from the test set to fill the training set (because we fill the whole column before the split).

Instead, we split the data into train and test sets first (still with missing values). Then calculate the `mean()` of the `Odometer (KM)` column on the training set and use it to fill the **training set** missing values *as well as* the **test set** missing values.

Now you might be asking, how does this happen?

Well, behind the scenes, `Pipeline()` calls a couple of methods:
1. `fit_transform()` - in our case, this computes the `mean()` of the `Odometer (KM)` column and then transforms the rest of the column on the **training data**. It also stores the `mean()` in memory.
2. `transform()` - uses the saved `mean()` of the `Odometer (KM)` column and transforms the **test** values.

Wait, wait, wait. This is confusing... how does the `Pipeline()` know what the training and test data are? We never told it?

You're right.

The magic trick is:
* `fit_transform()` is only ever used when calling `fit()` on your `Pipeline()` (in our case, when we used `model.fit(X_train, y_train)`.
* `transform()` is only ever used when calling `score()` or `predict()` on your `Pipeline()` (in our case, `model.score(X_test, y_test)`.

![what's happening with Pipeline](https://raw.githubusercontent.com/mrdbourke/zero-to-mastery-ml/master/images/sklearn-whats-happening-with-pipeline.png)

This means, when our missing **numerical values** get calculated and filled (using `fit_transform()`), they only happen on the training data (as long as you only pass `X_train` and `y_train` to `model.fit()`).

And since they get saved in memory, when we call `model.score(X_test, y_test)` and subsequently `transform()`, the test data gets preprocessed with information from the training set (using the past to try and predict the future, not the other way round).

### What about categorical values?

Since they usually don't depend on each other, categorical values are okay to be filled across sets and examples.

Okay, knowing all this, let's cross-validate our model pipeline using [cross_val_score()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html).

In [25]:
from sklearn.model_selection import cross_val_score

# Cross-validate our pipeline model
cross_val_score(model, X, y).mean()

0.22152985494597335

Since our `model` is an instance of `Pipeline()`, the same steps as we discussed above happen here with the `cross_val_score()`.

## Next steps

If you'd like to dig deeper into what's happening here, I'd suggest the following reading resources and next steps.
* **Reading:** [Scikit-Learn Pipeline() documentation](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).
* **Reading:** [Imputing missing values before building an estimator](https://scikit-learn.org/stable/auto_examples/impute/plot_missing_values.html) (compares different methods of imputing values).
* **Practice:** Try [tuning model hyperparameters with a `Pipeline()` and `GridSearchCV()`](https://scikit-learn.org/stable/modules/grid_search.html#composite-estimators-and-parameter-spaces).

How about all in 1 cell?

(the code below runs the same as the code above, just in 1 cell)

In [28]:
# import numpy as np
# import pandas as pd
# from sklearn.compose import ColumnTransformer
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.impute import SimpleImputer
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import OneHotEncoder

# # Import data from GitHub
# car_sales_missing = pd.read_csv("https://raw.githubusercontent.com/mrdbourke/zero-to-mastery-ml/master/data/car-sales-extended-missing-data.csv")

# # Drop the rows with no labels
# car_sales_missing.dropna(subset=["Price"], inplace=True)

# # Split data into X (data) & y (labels)
# X = car_sales_missing.drop("Price", axis=1)
# y = car_sales_missing["Price"]

# # Define categorical columns
# categorical_features = ["Make", "Colour"]
# # Create categorical transformer (imputes missing values, then one hot encodes them)
# categorical_transformer = Pipeline(steps=[
#   ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#   ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Define door feature
# door_feature = ["Doors"]
# # Create door transformer (fills all door missing values with 4)
# door_transformer = Pipeline(steps=[
#   ('imputer', SimpleImputer(strategy='constant', fill_value=4)),
# ])

# # Define numeric featrue
# numeric_features = ["Odometer (KM)"]
# # Create a transformer for filling all missing numeric values with the mean
# numeric_transformer = Pipeline(steps=[
#   ('imputer', SimpleImputer(strategy='mean'))
# ])

# # Create a column transformer which combines all of the other transformers
# # into one step
# preprocessor = ColumnTransformer(
#     transformers=[
#       ('categorical', categorical_transformer, categorical_features),
#       ('door', door_transformer, door_feature),
#       ('numerical', numeric_transformer, numeric_features)
# ])

# # Create the model pipeline
# model = Pipeline(steps=[('preprocessor', preprocessor), # this will fill our missing data and make sure it's all numbers
#                         ('model', RandomForestRegressor())]) # this will model our data
#                         #('regressor', RandomForestRegressor())]) # this will model our data. The regressor name here is very significant
#                         # because is the same name that we are going to use in the cell below example "regressor__max_features": ["sqrt"],

# # Split data into train and teset sets
# np.random.seed(42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# # Fit the model on the training data
# #(note: when fit() is called with a Pipeline(), fit_transform() is used for transformers)
# model.fit(X_train, y_train)

# # Score the model on the data
# # (note: when score() or  predict() is called with a Pipeline(), transform() is used for transformers)
# model.score(X_test, y_test)


0.22188417408787875

In [31]:
# Getting data ready
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Modelling
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

# Setup random seed
import numpy as np
np.random.seed(42)

# Import data and drop the rows with missing labels
data = pd.read_csv("data/car-sales-extended-missing-data.csv")
data.dropna(subset=["Price"], inplace=True)

# Define different features and transformer pipelines
categorical_features = ["Make", "Colour"]
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))])

door_feature = ["Doors"]
door_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value=4))])

numeric_features = ["Odometer (KM)"]
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))
])

# Setup preprocessing steps (fill missing values, then convert to numbers)
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
        ("door", door_transformer, door_feature),
        ("num", numeric_transformer, numeric_features)])

# Create a preprocessing and modelling pipeline
model = Pipeline(steps=[("preprocessor", preprocessor),
                        ("model", RandomForestRegressor(n_jobs=-1))])

# Split data
X = data.drop("Price", axis=1)
y = data["Price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Fit and score the model
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.22188417408787875

Documentation on ppipeline https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

It's also possible to use `GridSearchCV` or `RandomizedSesrchCV` with our Pipeline.

In [32]:
# Using grid search with pipeline
pipe_grid = {
    "preprocessor__num__imputer__strategy": ["mean", "median"], # note the double underscore after each prefix "preprocessor__"
    "model__n_estimators": [100, 1000],
    "model__max_depth": [None, 5],
    "model__max_features": ["sqrt"],
    "model__min_samples_split": [2, 4]
}

gs_model = GridSearchCV(model, pipe_grid, cv=5, verbose=2)
gs_model.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END model__max_depth=None, model__max_features=sqrt, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean; total time=   0.2s
[CV] END model__max_depth=None, model__max_features=sqrt, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean; total time=   0.2s
[CV] END model__max_depth=None, model__max_features=sqrt, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean; total time=   0.2s
[CV] END model__max_depth=None, model__max_features=sqrt, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean; total time=   0.2s
[CV] END model__max_depth=None, model__max_features=sqrt, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean; total time=   0.2s
[CV] END model__max_depth=None, model__max_features=sqrt, model__min_sampl

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Make',
                                                                          'Colour']),
                                                                        ('door',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value=4,
                                                                                                        strategy='constant'))]),
                                                                         ['Doors']),
                                                                        ('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer())]),
                                                                         ['Odometer '
                                                                          '(KM)'])])),
                                       ('model',
                                        RandomForestRegressor(n_jobs=-1))]),
             param_grid={'model__max_depth': [None, 5],
                         'model__max_features': ['sqrt'],
                         'model__min_samples_split': [2, 4],
                         'model__n_estimators': [100, 1000],
                         'preprocessor__num__imputer__strategy': ['mean',
                                                                  'median']},
             verbose=2)

In [33]:
model.score(X_test, y_test)

0.22188417408787875

What we've done is combine a series of data preprocessing steps (filling missing values, encoding numerical values) as well as a model into a Pipeline.

Doing so not only cleans up the code, it ensures the same steps are taken every time the code is run rather than having multiple different processing steps happening in different stages.

It's also possible to GridSearchCV or RandomizedSearchCV with a Pipeline.

The main difference is when creating a hyperparameter grid, you have to add a prefix to each hyperparameter (see the documentation for RandomForestRegressor for a full list of possible hyperparameters to tune).

The prefix is the name of the Pipeline step you'd like to alter, followed by two underscores.

For example, to adjust n_estimators of "model" in the Pipeline, you'd use: "model__n_estimators" (note the double underscore after model__ at the start).

In [35]:
# Score the best model
gs_model.score(X_test, y_test)

0.2848784564026805

Beautiful!

Using GridSearchCV we see a nice boost in our models score.

And the best thing is, because it's all in a Pipeline, we could easily replicate these results.

Where to next?
If you've made it this far, congratulations! We've covered a lot of ground in the Scikit-Learn library.

As you might've guessed, there's a lot more to be discovered.
     
But for the time being, you should be equipped with some of the most useful features of the library to start trying to apply them to your own problems.

Somewhere you might like to look next is to apply what you've learned above to a Kaggle competition. `https://www.kaggle.com/`

Kaggle competitions are great places to practice your data science and machine learning skills and compare your results with others.

A great idea would be to try to combine the heart disease classification code, as well as the Pipeline code, to build a model for the Titanic dataset.
`https://www.kaggle.com/c/titanic`

Otherwise, if you'd like to figure out what else the Scikit-Learn library is capable of I'd highly recommend browsing through 
the Scikit-Learn User Guide `https://scikit-learn.org/stable/user_guide.html` and seeing what sparks your interest.

Finally, as an extra-curriculum extension, you might want to look into trying out the CatBoost library `https://catboost.ai/`
for dealing with non-numerical data automatically.

The CatBoost algorithm is advanced version of a decision tree (like a Random Forest with superpowers) 
and is used in production at several large technology companies, including Cloudflare. `https://blog.cloudflare.com/how-cloudflare-runs-ml-inference-in-microseconds/`